<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
# Проверим наличие пропусков
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
# Поменяем тип данных в столбцах Возраст и Зарплата на целочисленный.
data['Возраст'] = pd.to_numeric(data['Возраст'], downcast='integer')
data['Зарплата'] = pd.to_numeric(data['Зарплата'], downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   int8   
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(1), int64(3), int8(1)
memory usage: 161.3 KB


Вывод:
Загрузили данные, поправили типы данных на целочисленные, где это необходимо. Пропусков нет

## Умножение матриц

Проверим, что вычисление линейной регрессии из Scikit-learn соответствует формуле.

In [7]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

In [8]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Коэффициенты регрессии полностью совпадают

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:** 

$$
a = Xw = X(X^T X)^{-1} X^T y = X X^{-1} (X^T)^{-1} X^T y = X X^{-1} y
$$

Заменим матрицу X на матрицу Z: 

$$
Z = XP (2.1)
$$

Пусть P при этом обратимая матрица, подходящей размерности для умножения.

Вычислим, чему будет равно предсказание и вектор весов.

$$
a1 = Zw1 (2.2)
$$

$$
w1 = (Z^T Z)^{-1} Z^T y
$$

Подставим значение w1 в уравнение 2.2, получим:

$$
a1 = Z (Z^T Z)^{-1} Z^T y
$$

Подставим вместо Z значение из 2.1:

$$
a1 = XP ((XP)^T (XP))^{-1} (XP)^T y
$$

$$
a1 = XP ((XP)^T (XP))^{-1} (XP)^T y = XP ( P^T X^T X P)^{-1} P^T X^T y = XP (P^T X^T X P)^{-1}  P^T X^T y
$$

$$
a1 = XP (P^T (X^T X) P)^{-1} P^T X^T y = XP P^{-1} (P^T (X^T X))^{-1} P^T X^T y = X (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

$$
a1 = X (X^T X)^{-1} X^T y = X w = a
$$


In [9]:

# w_P = P^{-1} w - это замена НУЖНАЯ

## Алгоритм преобразования

**Алгоритм**

Для защиты персональной информации на этапе поиска коэффициентов регрессии и предсказаний будем умножать матрицу признаков на обратимую матрицу, которую будет генерироваться случайным образом.

Этапы алгоритма:

Составление матрицы Y

Проверка матрицы на обратимость. Вычисление определителя матрицы Y

Получение матрицы преобразованных признаков Z = XY

Применение алгоритма на преобразованных признаках Z


**Обоснование:**

Матрица Y должна иметь необходимую размерность nxn, где n - количество признаков для регрессии. Таким образом, матрица Z будет иметь ту же размерность, что и матрица X. Обратная матрица существует, когда определитель не равен нулю.

## Проверка алгоритма

Проведем исследование модели по двум направлениям:

Исследуем качество модели без преобразования. 

1.1 С исходными признаками 

1.2 С отмасштабированными признаками

Исследуем качество модели с преобразованием. 

2.1 С исходными признаками 

2.2 С отмасштабированными признаками

Разделим данные на обучающие и тестовые.

In [10]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
prediction = model.predict(features_test)
R2_LR_origin_data = r2_score(target_test, prediction)
print("w-vector coef", model.coef_)
print("R2 =", R2_LR_origin_data)

w-vector coef [ 1.79258369e-02  3.57228278e-02 -5.46000708e-07 -1.26186590e-02]
R2 = 0.43522757127026546


In [12]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.4352275712702668


Модель ведет себя одинаково как на исходных данных, так и на отмасштабированных. 

Создадим функцию преобразования матрицы признаков.

In [13]:
def cipher_features(features):
    crypted_features = features
    n = features.shape[1]
    np.random.seed(12345)
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    while det == 0:
        cipher_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(cipher_matrix)
    crypted_features = crypted_features @ cipher_matrix
    return crypted_features, cipher_matrix

Присваивается переменной значения признаков, сохраняется размер для того, чтобы потом при генерации матрицы дать такой размер, чтобы можно было адекватно соразмерно умножить, далее генерируется случайная последовательность, создается случайная матрица, проверяется на обратимость, и пока она не будет обратима, повторяется генерация случайной последовательности и создается матрица, потом когда она все-таки получилась обратимой, преобразуем данные.

Выведем данные до преобразования и после.

In [14]:
old_features = features

In [15]:
display(features.head())
features, cipher_matrix = cipher_features(features)
display(features.head())
cipher_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0


array([[3, 6, 2, 5],
       [6, 3, 2, 7],
       [2, 8, 7, 1],
       [3, 2, 3, 7]])

In [16]:
# Для восстановления нужно закодированные данные умножить на обратную матрицу
new_old_features =  features @ np.linalg.inv(cipher_matrix) 

Данные потеряли свои начальные значения. 
Разобьем данные на тренировочную и обучающую выборку и проверим зачение R2

In [17]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [18]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_cipher_data)

w-vector coef [ 0.01280453  0.00328449 -0.00664342 -0.00538157]
R2 = 0.43522757127030764


In [19]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.43522757127026535


Сравним показатели качества моделей:

In [20]:
result = pd.DataFrame(data= [R2_LR_origin_data_scaled,
                      R2_LR_origin_data,
                      R2_LR_cipher_data,
                      R2_LR_cipher_data_scaled], 
                     columns=['R2'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабом',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабом',])
result

,R2
Линейная регрессия,0.435228
Линейная регрессия c масштабом,0.435228
Линейная регрессия на преобразованных признаках,0.435228
Линейная регрессия на преобразованных признаках c масштабом,0.435228


Вывод:

Были загружены и изучены данные

Качество линейной регресии не меняется при замене исходной матрицы на исходную матрицу, умноженную на обратимую.

Создан алгоритм преобразования данных с защитой персональных данных

Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с преобразованием

Можно сделать вывод, что данные очень просто зашифровать от распознавания, имея правильную матрицу.